In [ ]:
#https://github.com/jacobgil/pytorch-grad-cam

In [1]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50


ModuleNotFoundError: No module named 'pytorch_grad_cam'

In [ ]:

model = resnet50(pretrained=True)

#can try different target layers
#see model specific recommendations
# Choose the target layer you want to compute the visualization for.
    # Usually this will be the last convolutional layer in the model.
    # Some common choices can be:
    # Resnet18 and 50: model.layer4
    # VGG, densenet161: model.features[-1]
    # mnasnet1_0: model.layers[-1]
    # You can print the model to help chose the layer

target_layers = [model.layer4[-1]]
input_tensor = # Create an input tensor image for your model..
# Note: input_tensor can be a batch tensor with several images!

# We have to specify the target we want to generate the CAM for.
targets = [ClassifierOutputTarget(281)]

# Construct the CAM object once, and then re-use it on many images.
with GradCAM(model=model, target_layers=target_layers) as cam:
  # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
  grayscale_cam = cam(input_tensor=input_tensor, targets=targets)
  # In this example grayscale_cam has only one image in the batch:
  grayscale_cam = grayscale_cam[0, :]
  visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)
  # You can also get the model outputs without having to redo inference
  model_outputs = cam.outputs

In [ ]:
#Vikram code
#predict_additional.py

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

cam = GradCAM(model=model, target_layers=model.grad_cam_layer)

def run_image(
    img: Image.Image,
    model: PretrainedModel,
    preprocessing: v2.Compose,
    cam: GradCAM,
) -> list[float]:
    orig_img = pil_to_tensor(img)
    _, h, w = orig_img.shape
    img = orig_img[None]
    img = preprocessing(img)
    gradcam_output = cam(img)[0]
    gradcam_output = cv2.resize(gradcam_output, (w, h))
    gradcam_output = show_cam_on_image(
        orig_img.numpy().transpose((1, 2, 0)) / 255.0,
        gradcam_output,
        use_rgb=True,
        image_weight=0.7,
    )
    logits = model(img.to(device))
    return logits[0].tolist(), gradcam_output

In [2]:
#Bing
#how to use grad cam in pytorch

import torch
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from torchvision.models import resnet50, ResNet50_Weights

In [5]:
#model = models.resnet50(pretrained=True)
#model.eval()
#Import model for training

#ResNet50 
weights = ResNet50_Weights.DEFAULT
model = torchvision.models.resnet50(weights=weights)
model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [7]:
def preprocess_image(img_path):
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img = Image.open(img_path).convert('RGB')
    img_tensor = preprocess(img).unsqueeze(0)
    return img_tensor

#path to image
img_tensor = preprocess_image('/project/90daydata/nematode_ml/BLD/NematodeDataset/train/BLD/DSC07461 - Copy.jpg')


In [8]:
gradients = []

def save_gradient(grad):
    gradients.append(grad)

# Assuming the last convolutional layer is layer4[2].conv3 in ResNet50
target_layer = model.layer4[2].conv3
target_layer.register_backward_hook(lambda module, grad_input, grad_output: save_gradient(grad_output[0]))


In [9]:
output = model(img_tensor)
pred_class = output.argmax().item()

# Zero the gradients
model.zero_grad()

# Backward pass
output[0, pred_class].backward()


/project/nematode_ml/envs/ipykernel/lib/python3.8/site-packages/torch/nn/modules/module.py:1640: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


In [10]:
# Get the gradients and activations
gradients = gradients[0].cpu().data.numpy()
activations = target_layer.output.cpu().data.numpy()[0]

# Compute the weights
weights = np.mean(gradients, axis=(1, 2))

# Compute the Grad-CAM
grad_cam = np.zeros(activations.shape[1:], dtype=np.float32)
for i, w in enumerate(weights):
    grad_cam += w * activations[i, :, :]

# Apply ReLU
grad_cam = np.maximum(grad_cam, 0)

# Normalize the Grad-CAM
grad_cam = grad_cam / np.max(grad_cam)
grad_cam = np.uint8(255 * grad_cam)
grad_cam = np.resize(grad_cam, (224, 224))


AttributeError: 'Conv2d' object has no attribute 'output'

In [ ]:
def show_cam_on_image(img_path, mask):
    img = Image.open(img_path).convert('RGB')
    img = np.array(img.resize((224, 224)))
    heatmap = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    cam = heatmap + np.float32(img) / 255
    cam = cam / np.max(cam)
    plt.imshow(cam)
    plt.show()

show_cam_on_image('path_to_your_image.jpg', grad_cam)
